In [ ]:
!pip install PyPDF2
!pip install tiktoken
!pip install --upgrade langchain -q
!pip install "openai<1.0.0"
#!pip install openai==1.3.7
!pip install sentence_transformers -q
!pip install unstructured -q
!pip install tabulate
!pip install boto3
!pip install pinecone-client
!pip install pymupdf
!pip install faiss-gpu

In [ ]:
import boto3
from PyPDF2 import PdfReader
from io import BytesIO
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import fitz  # PyMuPDF
import tempfile
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback

In [ ]:
# AWS credentials (replace these with your credentials)
aws_access_key_id = '---'
aws_secret_access_key = '---'

# Bucket and file details
bucket_name = '---'
file_key = '---'

In [ ]:
s3 = boto3.client('s3', 
                  aws_access_key_id=aws_access_key_id, 
                  aws_secret_access_key=aws_secret_access_key)

In [ ]:
# Read the file from S3 and save it to a temporary location
try:
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    
    # Get the response body stream
    response_body = response['Body']
    
    # Create a temporary file to save the PDF
    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(response_body.read())
        temp_file_path = temp_file.name
    
    # Open the PDF file with PyMuPDF
    pdf_document = fitz.open(temp_file_path)
    
    # Extract text from each page
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    
    # Close the PDF document
    pdf_document.close()
    
    # Remove the temporary file
    os.unlink(temp_file_path)
    
    print(text)
except Exception as e:
    print(f"Error reading PDF file from S3: {e}")

In [ ]:
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=150,
        length_function=len
    )
chunks = text_splitter.split_text(text)

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
vec_db = FAISS.from_texts(chunks, embeddings)

In [ ]:
os.environ['OPENAI_API_KEY']="---"

In [ ]:
llm = OpenAI()

In [ ]:
query = "---" #Add the User question
res_1 = vec_db.similarity_search(query)

In [ ]:
prompt = f"""
You are a text summarizer.
Follow exactly those 3 steps:
1. Read the context below and answer the user question
Context : {res_1}
2. Answer the question using only this context
3. If you dont know the answer, reply "Need More Context"
User Question: {query}

If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
"""

In [ ]:
act = llm(PROMPT)

In [ ]:
act